In [1]:
import pandas as pd
from python_files.constants import KEYPHRASES
from python_files.objs import get_datetime, get_participant, get_message_type

In [2]:
df = pd.DataFrame(
    columns=[
        "datetime", "participant", "type", "message",
        "edited", "calls_meta", "deleted", "view_once"
    ]
)

In [15]:
i = 0

with open("./data/analysis/chat.txt", "r") as file:
    lines = file.readlines()
    for cont, line in enumerate(lines):
        line: list[str, str] = line.split(" - ", 1)
        if len(line) == 1:
            continue

        datetime, message = line

        # Calcular variables
        datetime = get_datetime(datetime)
        participant = get_participant(message)
        message = message.removeprefix(f"{participant}: ").strip()  # Mensaje sin el nombre del participante
        message_type = get_message_type(message)  # Tipo de mensaje
        edited = message.endswith(KEYPHRASES["edit"])  # Mensaje editado
        deleted = message == KEYPHRASES["delete"]  # Mensaje eliminado
        view_once = message == KEYPHRASES["view_once"]  # Mensaje de vista única
        calls_metaai = message.startswith(KEYPHRASES["meta"])  # Llamada a MetaAI

        if edited:
            message = message.removesuffix(KEYPHRASES["edit"]).strip()

        if message_type is None:
            # Tipo de mensaje nulo (eliminado o para una vez). Se hace así para no generar problemas luego
            df.loc[i] = [datetime, participant, message_type, message, edited, calls_metaai, deleted, view_once]
            i += 1
            continue

        if message_type == "file":
            # Colocar el nombre del archivo en el df, y una línea para el subtítulo del archivo (si la hay)
            message = message.removesuffix(KEYPHRASES["file"]).strip()
            j = cont + 1
            suptitle = ""
            while len(lines[j].split(" - ", 1)) == 1:
                suptitle = lines[j].strip()
                j += 1
            
            df.loc[i] = [datetime, participant, "file", message, edited, False, False, False]
            i += 1

            if suptitle:
                df.loc[i] = [datetime, participant, "message", suptitle, edited, calls_metaai, False, False]
                i += 1
            continue

        if message_type == "poll":
            # Colocar todas las opciones en un texto, separadas por comas
            # y un texto aparte para el titulo de la encuesta
            options = []
            title_poll = ""
            j = cont + 1
            while len(lines[j].split(" - ", 1)) == 1 and len(lines[j].strip()) > 0:
                if lines[j].startswith(KEYPHRASES["option"]):
                    options.append(lines[j].removeprefix(KEYPHRASES["option"]).strip())
                else:
                    title_poll = lines[j].strip()
                j += 1

            options_str = ",".join(options)
            df.loc[i] = [datetime, participant, "message", title_poll, edited, False, False, False]
            df.loc[i+1] = [datetime, participant, "poll", options_str, False, False, False, False]
            i += 2
            continue

        message = message.strip()
        df.loc[i] = [datetime, participant, message_type, message, edited, calls_metaai, deleted, view_once]
        i += 1

In [12]:
df

,datetime,participant,type,message,edited,calls_meta,deleted,view_once
0,2024-02-16 20:13:00,Mariana,message,Oigan,False,False,False,False
1,2024-02-16 20:13:00,Mariana,message,Vengan a mi U,False,False,False,False
2,2024-02-16 20:13:00,Negron,message,Apoyo,False,False,False,False
3,2024-02-16 20:13:00,Negron,message,Apoyo,False,False,False,False
4,2024-02-16 20:13:00,Negron,message,Apoyo,False,False,False,False
...,...,...,...,...,...,...,...,...
59100,2025-02-23 09:05:00,Mariana,message,"No, hinchazón",False,False,False,False
59101,2025-02-23 09:06:00,Mariana,message,Wow,False,False,False,False
59102,2025-02-23 09:06:00,Mariana,message,Buenos días,False,False,False,False
59103,2025-02-23 10:01:00,JuanD,message,Buenos días,False,False,False,False


In [18]:
df.to_csv("./data/analysis/chat.csv")